# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json


# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# load and print city dataframe
city_df= pd.read_csv('output_data/cities.csv')
city_df.head()


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,busselton,-33.65,115.33,11.11,68,0,4.45,AU,1602875405
1,severo-kurilsk,50.68,156.12,5.29,72,75,7.81,RU,1602875299
2,meulaboh,4.14,96.13,24.95,85,100,1.71,ID,1602875405
3,kodiak,57.79,-152.41,2.00,74,1,2.24,US,1602875405
4,punta arenas,-53.15,-70.92,5.00,86,90,5.10,CL,1602875405


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
# configure the map
gmaps.configure(api_key = g_key)

# set the zoom and center on the map
fig = gmaps.figure(zoom_level=1.8,center=(0,0))

# assign coordinates and humitday to map
locations = city_df[["Lat","Lng"]]
humidity = city_df["Humidity"]

# assign the heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating = False,
                                max_intensity = 10, point_radius=1)

# add the heat layer to the figure
fig.add_layer(heat_layer)

#display the figure
fig



Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [8]:

# reduce the cites with extreme tempearatures, high humidity and high cloudiness
vac_df = city_df[(city_df["Humidity"]<20) & (city_df["Cloudiness"]<10) & 
                 (city_df["Max Temp"]>24) & (city_df["Max Temp"]<36)] 

# remove cities with null values
vac_df = vac_df.dropna()

# print cities ideal for vacation
vac_df


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
132,magalia,39.81,-121.58,30.00,12,5,6.20,US,1602875260
145,bilma,18.69,12.92,32.01,14,0,2.87,NE,1602874823
153,tessalit,20.20,1.01,33.35,11,0,3.36,ML,1602874866
159,adrar,20.50,-10.07,34.35,13,0,4.15,MR,1602874866
187,half moon bay,37.46,-122.43,31.00,12,1,9.80,US,1602874858
204,manyana,-23.40,21.72,26.70,19,0,3.99,BW,1602875269
224,umm kaddadah,13.60,26.69,33.15,11,0,5.71,SD,1602875271
345,werda,-25.27,23.28,29.00,15,0,1.91,BW,1602875287
346,tanout,14.97,8.89,34.20,15,0,4.37,NE,1602875287
368,diego de almagro,-26.37,-70.05,30.26,13,0,6.01,CL,1602875289


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
# ----- THis is the one ----

# reset the index for looping
vac_df.reset_index(drop=True, inplace=True)

# create empty lists to append to
lats = []
lngs = []

# get lattitudes
for item in vac_df["Lat"]:
        lats.append(item)

#get longitudes
for item in vac_df["Lng"]:
        lngs.append(item)  


# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# create empty list to append to
hotel_list=[]

# set search parameters
target_radius = 5000
target_type = "hotel"

# loop through cities
for item in vac_df.index:
    # get a hotel for each city
    try:   
        # geocoordinates
        target_coordinates = f"{lats[item]},{lngs[item]}"
       
        # set up a parameters dictionary
        params = {
                "location":target_coordinates,
                "radius": target_radius,
                "type": target_type,
                "key": g_key
                 }     
        
        # run a request using the params dictionary
        response = requests.get(base_url, params=params)

        # convert response to json
        places_data = response.json()
        
        # add hotel name to list
        hotel_list.append(places_data["results"][1]["name"])
    # prevent break    
    except:
        # if no hotel found, add No Hotel to list
        hotel_list.append("No Hotel")

# add the hotels to the dataframe        
vac_df["Hotel Name"] = hotel_list

# create no dataframe with only cities we found hotels in
hotel_df = vac_df[vac_df['Hotel Name'] != "No Hotel"]


In [25]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [29]:
# create markers for map
markers = gmaps.marker_layer(locations,
    info_box_content=hotel_info)

# Add marker layer ontop of heat map
fig.add_layer(markers)

# Display figure
fig



Figure(layout=FigureLayout(height='420px'))

It looks like the climate I am lookng for is a certain range of lattitudes.  The absolute value (distance from the equator) of cities with my ideal weather is around 25-35 degrees latitude.